In [1]:
from strategies import random_nodes, learn_target_parents, edge_prob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from config import DATA_FOLDER
from strategies.simulator import GenerationConfig, SimulationConfig, simulate
from analysis.check_gies import check_gies
%matplotlib inline
sns.set()
np.random.seed(1729)

In [2]:
N_NODES = 50
DAG_FOLDER = os.path.join(DATA_FOLDER, 'medium')
STRATEGIES = {
    'random': random_nodes.random_strategy,
    'learn-parents': learn_target_parents.create_learn_target_parents(N_NODES - 1, 100),
    'edge-prob': edge_prob.create_edge_prob_strategy(N_NODES-3, 100)
}

### Generate and save DAGs

In [3]:
G_CONFIG = GenerationConfig(
    n_nodes=N_NODES,
    edge_prob=.5,
    n_dags=3
)
gdags = G_CONFIG.save_dags(DAG_FOLDER)

=== Saving DAGs ===
=== Saved ===


In [4]:
[str(g) for g in gdags]

['[1][3|1][2|1][8|2,3][0][6|0,1][5|0,1,3][4|0,1,3][10|0,4,5,6,8][11|1,2,5,6,8,10][13|0,2,3,8,10,11][7|2,3,4,6][9|0,4,6,7][12|0,1,3,6,7,8,9][14|1,6,9,10,11,12][15|1,2,3,4,5,6,7,8,10,11,14][18|0,1,3,5,6,7,9,10,11,12,13,14][17|0,2,3,7,9,10,12,14][16|0,1,3,4,7,10,11,14][19|0,1,2,3,4,7,9,16,18][20|0,2,5,8,10,16,18,19][26|5,6,8,10,11,12,14,15,16,20][22|0,1,3,4,8,9,10,11,13,16,17,20][24|1,3,4,5,6,8,16,17,20,22][21|1,2,3,9,10,12,13,15,16,19,20][25|3,5,6,10,11,13,16,17,20,21,24][28|1,3,7,8,12,15,17,19,22,24,25][23|1,2,4,5,6,7,8,12,14,15,17,21,22][27|2,3,4,7,8,13,16,18,20,21,23,25,26][30|4,5,7,8,9,10,12,13,14,15,18,19,20,22,23,25,26,27,28][29|3,8,10,16,18,19,20,21,22,23,25,27][31|0,1,4,5,8,10,11,13,15,16,19,21,29,30][32|1,2,3,5,6,14,15,19,21,25,27,28,29,30][33|0,1,2,3,4,32,6,8,9,12,14,17,19,20,22,23,26,31][35|0,33,1,2,4,8,10,12,13,17,18,19,21,24][36|0,33,2,35,32,3,6,5,8,9,10,1,15,18,20,22,24,25][37|1,2,33,4,5,7,8,9,12,14,15,16,18,19,20,24,26,27,29][34|32,4,5,6,7,8,10,11,13,14,15,16,18,20,21,23,2

### Run random strategy on each DAG

In [5]:
SIM_CONFIG_RANDOM = SimulationConfig(
    starting_samples=250,
    n_samples=50,
    n_batches=2,
    max_interventions=2,
    strategy='random',
    intervention_strength=2,
)

In [6]:
for i, gdag in enumerate(gdags):
    print('=== Simulating strategy for DAG %d' % (i))
    simulate(STRATEGIES['random'], SIM_CONFIG_RANDOM, gdag, os.path.join(DAG_FOLDER, 'dag%d' % i, 'random'))

=== Simulating strategy for DAG 0
/Users/chandlersquires/Documents/active_learning/new/data/medium/dag0/random/samples
Batch 0


ValueError: n_samples / (n_batches * max interventions) must be an integer

### Run edge-prob strategy on each DAG

In [ ]:
SIM_CONFIG_EDGE_PROB = SimulationConfig(
    starting_samples=250,
    n_samples=50,
    n_batches=2,
    max_interventions=2,
    strategy='edge-prob',
    intervention_strength=2,
)

In [ ]:
for i, gdag in enumerate(gdags):
    print('=== Simulating strategy for DAG %d' % (i))
    simulate(STRATEGIES['edge-prob'], SIM_CONFIG_EDGE_PROB, gdag, os.path.join(DAG_FOLDER, 'dag%d' % i, 'edge-prob'))

### Run learn-parents strategy on each DAG

In [ ]:
SIM_CONFIG_PARENTS = SimulationConfig(
    starting_samples=250,
    n_samples=50,
    n_batches=2,
    max_interventions=2,
    strategy='learn-parents',
    intervention_strength=2,
)

In [ ]:
for i, gdag in enumerate(gdags):
    print('=== Simulating strategy for DAG %d' % (i))
    simulate(STRATEGIES['learn-parents'], SIM_CONFIG_EDGE_PROB, gdag, os.path.join(DAG_FOLDER, 'dag%d' % i, 'learn-parents'))

### Check gathered data

In [ ]:
random_rhs = []
for d, gdag in enumerate(gdags):
    folder = os.path.join(DAG_FOLDER, 'dag%d' % d)
    parent_probs, rh = check_gies(folder, 'random', N_NODES-3)
    random_rhs.append(rh)

In [ ]:
random_tprs = [rh.tpr for rh in random_rhs]
random_fprs = [rh.fpr for rh in random_rhs]
print(random_tprs)
print(random_fprs)

In [ ]:
edge_prob_rhs = []
for d, gdag in enumerate(gdags):
    folder = os.path.join(DAG_FOLDER, 'dag%d' % d)
    parent_probs, rh = check_gies(folder, 'edge-prob', N_NODES-3)
    edge_prob_rhs.append(rh)

In [ ]:
edge_prob_tprs = [rh.tpr for rh in edge_prob_rhs]
edge_prob_fprs = [rh.fpr for rh in edge_prob_rhs]
print(edge_prob_tprs)
print(edge_prob_fprs)

In [ ]:
learn_parents_rhs = []
for d, gdag in enumerate(gdags):
    folder = os.path.join(DAG_FOLDER, 'dag%d' % d)
    parent_probs, rh = check_gies(folder, 'learn-parents', N_NODES-3)
    learn_parents_rhs.append(rh)

In [ ]:
learn_parents_tprs = [rh.tpr for rh in learn_parents_rhs]
learn_parents_fprs = [rh.fpr for rh in learn_parents_rhs]
print(learn_parents_tprs)
print(learn_parents_fprs)